In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
##define variables

In [2]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soupa = BeautifulSoup(page.text, 'html.parser')

In [3]:
records = []

# table recherche
code_table = soupa.find('table')

# p items
'''
<p>
    <b>M4A</b><br/>
    <span style="font-size:80%;">
        <a href="/wiki/North_York" title="North York">North York</a>
        <br/>
        (<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)
    </span>
</p>,

'''
name_codes_item = code_table.find_all('p')

for mysource in name_codes_item:    
    
    myPostaCode = mysource.find('b').contents[0]
    # <b>M4A</b><br/> -> M4A
    
   
    myBorough_myNeighborhood = mysource.find_all('span')[0].text
    # <a href="/wiki/North_York" title="North York">North York</a> (<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)
    
    first_part = myBorough_myNeighborhood.replace( ')', '') 
    first_part1 = first_part.replace( '/', ', ')
    #first_part1
    new_base = first_part1.split('(')
    
    # North York
    myBorough = new_base[0:1]
    myBorough = ','.join(myBorough)
    
    # Victoria Village
    myNeighborhood = new_base[1:]
    myNeighborhood = ','.join(myNeighborhood) 
    if myNeighborhood == '':
        myNeighborhood = myBorough # clone Borough's value if no value to Neighborhood
    else:
        myNeighborhood = myNeighborhood
    
    
    #print( myPostaCode, myBorough,myNeighborhood )
    records.append((myPostaCode,myBorough, myNeighborhood))

In [ ]:
## define df for the city

In [4]:
import pandas as pd
df_toronto = pd.DataFrame(records, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [ ]:
##check df

In [5]:
df_toronto.head(10)

,PostalCode,Borough,Neighborhood


In [ ]:
#clean data

In [6]:
# delete "Not assigned"
import numpy as np
df_toronto.replace("Not assigned", np.nan, inplace=True)

,PostalCode,Borough,Neighborhood


In [7]:
df_toronto.dropna(inplace=True)

In [8]:
# re Index
df_toronto.reset_index(drop = True, inplace = True)

In [9]:
df_toronto

,PostalCode,Borough,Neighborhood


In [11]:
# -> Queen's Park\n  -> Queen's Park
# -> M5E  Downtown Toronto          Stn A PO Boxes25 The Esplanade                  Enclave of M5E -> Downtown Toronto
# -> M3C North York       (Don Mills) South (Flemingdon Park)
# -> M7Y  East Toronto Business      reply mail Processing Centre 969 Eastern       (Enclave of M4L)-> East Toronto
# -> M7R	MississaugaCanada Post Gateway Processing Centre Enclave of L4W         -> Mississauga
# -> M7Y	East TorontoBusiness reply mail Processing Cen...	East Toronto -> East Toronto
newdf_toronto =df_toronto.replace({
    'Queen\'s Park\n': 'Queen\'s Park',
    'Enclave of M4L': 'East Toronto',
    'Enclave of M5E': 'Downtown Toronto',
    'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto',
    'Enclave of L4W':'East Toronto',
    'MississaugaCanada Post Gateway Processing Centre': 'Mississauga',
    'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto'
})
newdf_toronto

,PostalCode,Borough,Neighborhood


In [12]:
len(newdf_toronto)

0

In [15]:
np.shape(newdf_toronto)

(0, 3)

In [16]:
newdf_toronto.to_csv('toronto_base.csv')